In [ ]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cvxEDA.src.cvxEDA as cvxEDA
from itertools import product
from tqdm import tqdm
from scipy.stats import pearsonr
from scipy.stats import skew
from scipy.stats import kurtosis
import warnings

# Use the warnings filter to suppress specific warnings
warnings.filterwarnings("ignore")

In [ ]:
from scipy.stats.mstats import winsorize
import neurokit2 as nk
import heartpy as hp

In [ ]:
Label = ['0_Baseline_Baseline','V1_LVLA_The Displaced', 'V2_LVLA_Happyland 360',
       'V3_LVHA_Jailbreak 360', 'V4_LVHA_War Knows No Nation',
       'V5_HVHA_Canyon Swing', 'V6_HVLA_Redwoods Walk Among Giants',
       'V7_HVHA_Speed Flying', 'V8_HVLA_Instant Caribbean Vacation',
       'V1_LVLA_The Nepal Earthquake Aftermath',
       'V2_LVHA_Zombie Apocalypse Horror', 'V3_LVLA_Abandoned building',
       'V4_LVHA_Kidnapped', 'V5_HVHA_Mega Coaster',
       'V6_HVLA_Malaekahana Sunrise',
       'V7_HVHA_Puppies host SourceFed for a day',
       'V8_HVLA_Great Ocean Road', 'V1_LVHA_War Knows No Nation',
       'V2_HVLA_Redwoods Walk Among Giants', 'V3_LVLA_Happyland 360',
       'V4_HVHA_Speed Flying', 'V5_HVLA_Instant Caribbean Vacation',
       'V6_LVHA_Jailbreak 360', 'V7_LVLA_The Displaced',
       'V8_HVHA_Canyon Swing', 'V1_LVHA_Kidnapped',
       'V2_HVLA_Malaekahana Sunrise', 'V3_LVHA_Zombie Apocalypse Horror',
       'V4_HVHA_Puppies host SourceFed for a day',
       'V5_HVLA_Great Ocean Road', 'V6_LVLA_Abandoned building',
       'V7_LVLA_The Nepal Earthquake Aftermath', 'V8_HVHA_Mega Coaster']

lst = ['p1','p3','p2','p4']

fs_dict = {'EDA': 15.625, 'PPG': 125}

In [ ]:
vads = pd.read_csv("VADS.csv")
vads

# Data Cleaning

In [ ]:
ppg_df = pd.DataFrame()
eda_df = pd.DataFrame()

In [ ]:
for playlist in lst:
    temp1 = pd.read_csv(f"PPG/{playlist}.csv")
    temp1['playlist_id']=playlist
    ppg_df = ppg_df.append(temp1,ignore_index=True)
ppg_df

In [ ]:
ppg_df['id'].unique().shape

In [ ]:
ppg_df[ppg_df['id']=='p18']['label'].unique()

In [ ]:
for playlist in lst:
    temp1 = pd.read_csv(f"EDA/{playlist}.csv")
    temp1['playlist_id']=playlist
    eda_df = eda_df.append(temp1,ignore_index=True)
eda_df

In [ ]:
eda_df['id'].unique().shape

In [ ]:
eda_df[eda_df['id']=='p18']['label'].unique()

In [ ]:
#removing error, discard and interview data
print(ppg_df['label'].unique().shape)
ppg_df['label'].unique()

In [ ]:
#removing error, discard and interview data
print(eda_df['label'].unique().shape)
eda_df['label'].unique()

In [ ]:
rest_lst = ['rest1', 'rest2', 'rest3', 'rest4', 'rest5', 'rest6', 'rest7', 'rest8', 'rest', 'rest9']
 
error_list = ['error', 'error_eda', 'errorcheck', 'discard1', 'discard2','eda disconnected', 'error1', 'Segment 1']

vr_familiarity = ['vrfamiliarity', 'vrfamiliriaty', 'VR Familiarity', 'vrfamiarity', 'vrfamirity']

baseline = ['baseline', 'baseline ', 'baseline 2']

extra_data = ['rollercoaster', 'interview']  

eda_df['label'] = eda_df['label'].replace(rest_lst, 'rest')
eda_df['label'] = eda_df['label'].replace(error_list, 'error')
eda_df['label'] = eda_df['label'].replace(vr_familiarity, 'VRfamiliarity')
eda_df['label'] = eda_df['label'].replace(baseline, '0_Baseline_Baseline')
eda_df['label'] = eda_df['label'].replace(extra_data, 'rest')
eda_df['label'] = eda_df['label'].replace(' V8_HVLA_Great Ocean Road', 'V8_HVLA_Great Ocean Road')
eda_df['label'] = eda_df['label'].replace('V5_HVHA_Canyon Swing_1', 'V5_HVHA_Canyon Swing')
eda_df['label'] = eda_df['label'].replace('V2_HVLA_Redwoods Walk Among Giants,', 'V2_HVLA_Redwoods Walk Among Giants')
eda_df['label'] = eda_df['label'].replace('V2_HVLA_Redwoods Walk Among Giants_1', 'V2_HVLA_Redwoods Walk Among Giants')
eda_df['label'] = eda_df['label'].replace('V8_HVHA_Mega Coaster_1', 'V8_HVHA_Mega Coaster')

eda_df['label'].unique().shape

In [ ]:
eda_df['label'].unique()

In [ ]:
ppg_df['label'] = ppg_df['label'].replace(rest_lst, 'rest')
ppg_df['label'] = ppg_df['label'].replace(error_list, 'error')
ppg_df['label'] = ppg_df['label'].replace(vr_familiarity, 'VRfamiliarity')
ppg_df['label'] = ppg_df['label'].replace(baseline, '0_Baseline_Baseline')
ppg_df['label'] = ppg_df['label'].replace(extra_data, 'rest')
ppg_df['label'] = ppg_df['label'].replace(' V8_HVLA_Great Ocean Road', 'V8_HVLA_Great Ocean Road')
ppg_df['label'] = ppg_df['label'].replace('V5_HVHA_Canyon Swing_1', 'V5_HVHA_Canyon Swing')
ppg_df['label'] = ppg_df['label'].replace('V2_HVLA_Redwoods Walk Among Giants,', 'V2_HVLA_Redwoods Walk Among Giants')
ppg_df['label'] = ppg_df['label'].replace('V2_HVLA_Redwoods Walk Among Giants_1', 'V2_HVLA_Redwoods Walk Among Giants')
ppg_df['label'] = ppg_df['label'].replace('V8_HVHA_Mega Coaster_1', 'V8_HVHA_Mega Coaster')

In [ ]:
ppg_df['label'].unique().shape

In [ ]:
ppg_df['label'].unique()

In [ ]:
temp_df = pd.DataFrame(eda_df)

# Splitting the label column based on separator '_'
separator = '_'
split_columns = temp_df['label'].str.split(separator, expand=True)
# print(split_columns)

# Rename the new columns
new_columns = ['Video ID', 'CMA', 'Video Name']
split_columns.columns = new_columns

# Combine the split columns with the original DataFrame
eda_df = pd.concat([eda_df, split_columns], axis=1)
eda_df

In [ ]:
temp_df1 = pd.DataFrame(ppg_df)

# Splitting the label column based on separator '_'
separator = '_'
split_columns1 = temp_df1['label'].str.split(separator, expand=True)
# print(split_columns1)

# Rename the new columns
new_columns1 = ['Video ID', 'CMA', 'Video Name']
split_columns1.columns = new_columns1

# Combine the split columns with the original DataFrame
ppg_df = pd.concat([ppg_df, split_columns1], axis=1)
ppg_df

In [ ]:
eda_df.columns = ['Time', 'EDA', 'Label','Participant ID' , 'Gender', 'Playlist ID', 'Video ID', 'CMA', 'VideoName'] 
eda_df

In [ ]:
eda_df['Participant ID'].unique().shape

In [ ]:
ppg_df.columns = ['Time', 'BPM', 'PPG', 'IBI', 'Label','Participant ID' , 'Gender', 'Playlist ID', 'Video ID', 'CMA', 'VideoName']
ppg_df

In [ ]:
ppg_df['Participant ID'].unique().shape

In [ ]:
eda_df['Participant ID'] = eda_df['Participant ID'].apply(lambda x: int(x.split('p')[1]) if x.startswith('p') else x)
eda_df['Participant ID'].unique()

In [ ]:
eda_df['Playlist ID'] = eda_df['Playlist ID'].apply(lambda x: int(x.split('p')[1]) if x.startswith('p') else x)
eda_df['Playlist ID'].unique()

In [ ]:
eda_df['Gender'] = eda_df['Gender'].apply(lambda x: 'Male' if x.startswith('b') else 'Female')
eda_df

In [ ]:
eda_df['Participant ID'].unique()

In [ ]:
ppg_df['Participant ID'] = ppg_df['Participant ID'].apply(lambda x: int(x.split('p')[1]) if x.startswith('p') else x)
ppg_df['Playlist ID'] = ppg_df['Playlist ID'].apply(lambda x: int(x.split('p')[1]) if x.startswith('p') else x)
ppg_df['Gender'] = ppg_df['Gender'].apply(lambda x: 'Male' if x.startswith('b') else 'Female')
ppg_df['Participant ID'].unique()

In [ ]:
ppg_df

In [ ]:
# code to replace anamoly with participant average value
min_bpm = 35
max_bpm = 140
# Group the DataFrame by participant
grouped = ppg_df.groupby('Participant ID')

# Replace values outside the range with participant-wise average
for participant, group_df in grouped:
#     print(participant)
    filtered_group = group_df[(group_df['BPM'] >= min_bpm) & (group_df['BPM'] <= max_bpm)]
    average_bpm = filtered_group['BPM'].mean()
    ppg_df.loc[(ppg_df['Participant ID'] == participant) &
                ((ppg_df['BPM'] < min_bpm) | (ppg_df['BPM'] > max_bpm)), 'BPM'] = average_bpm

In [ ]:
ppg_df['BPM'].describe()

In [ ]:
ppg_df.to_csv("PPG.csv",index=False)

In [ ]:
# code to replace anamoly with participant average value
min_EDA = 0
max_EDA = 60
# Group the DataFrame by participant
grouped = eda_df.groupby('Participant ID')

# Replace values outside the range with participant-wise average
for participant, group_df in grouped:
#     print(participant)
    filtered_group = group_df[(group_df['EDA'] >= min_EDA) & (group_df['EDA'] <= max_EDA)]
    eda_df.loc[(eda_df['Participant ID'] == participant) &
                ((eda_df['EDA'] < min_EDA) | (eda_df['EDA'] > max_EDA)), 'EDA'] = 0

In [ ]:
eda_df['EDA'].describe()

In [ ]:
eda_df.to_csv("EDA.csv",index=False)